In [1]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd

## Импортируем dataset

In [2]:
url='https://lms.skillfactory.ru/assets/courseware/v1/c903ecd0b0c995c44213d620ab6ae94d/asset-v1:SkillFactory+MIPTDS+SEPT22+type@asset+block/churn.zip'
churn_data = pd.read_csv(url)
churn_data.head() 

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Столбцы таблицы:

- RowNumber — номер строки таблицы (это лишняя информация, поэтому можете сразу от неё избавиться)
- CustomerId — идентификатор клиента
- Surname — фамилия клиента
- CreditScore — кредитный рейтинг клиента (чем он выше, тем больше клиент брал кредитов и возвращал их)
- Geography — страна клиента (банк международный)
- Gender — пол клиента
- Age — возраст клиента
- Tenure — сколько лет клиент пользуется услугами банка
- Balance — баланс на счетах клиента в банке
- NumOfProducts — количество услуг банка, которые приобрёл клиент
- HasCrCard — есть ли у клиента кредитная карта (1 — да, 0 — нет)
- IsActiveMember — есть ли у клиента статус активного клиента банка (1 — да, 0 — нет)
- EstimatedSalary — предполагаемая заработная плата клиента
- Exited — статус лояльности (1 — ушедший клиент, 0 — лояльный клиент)

→ В файле должно содержаться 10 графиков — 10 ответов к заданиям.

→ Каждый график и преобразования к нему выполняются в отдельной ячейке.

→ Под графиком вы должны предоставить свой ответ на вопрос по нему и, если это требуется, выводы, которые вы можете сделать, исходя из графика.

# ЗАДАНИЯ

Для решения итоговых заданий по визуализации вы можете сами выбирать, какую библиотеку использовать. Если у вас возникнут трудности при решении задач, рекомендуем обратиться к документации по библиотекам [Matplotlib](https://matplotlib.org/), [Seaborn](https://seaborn.pydata.org/) и [Plotly](https://plotly.com/python/), где собраны сотни различных примеров построения графиков — вы точно найдёте то, что вам нужно. Также предлагаем вам обратиться к специальной литературе по визуализации:

- [Шпаргалка по визуализации данных в Plotly](https://habr.com/ru/post/502958/)  на Хабре;
- [Статья от SkillFactory по продвинутой визуализации данных с Plotly](https://habr.com/ru/company/skillfactory/blog/510320/) на Хабре
- [Книга по визуализации данных](https://drive.google.com/file/d/1slYxv6hG4pdUwyGiYiSIOnI6rgk2fyky/view?usp=share_link) в Matplotlib и Seaborn;

## 9.1. Каково соотношение ушедших и лояльных клиентов?
Покажите это на графике и дайте комментарий по соотношению.

In [3]:
fig = go.Figure()


exited_percent = churn_data['Exited'].value_counts()
pull = [0]*len(churn_data['Exited'])

to_drow = go.Pie(values=exited_percent, labels=['Остались', 'Ушли'], pull=pull, hole=0.7)


# отрисовка
fig.add_trace(to_drow)

# подписи
fig.update_layout(template = 'plotly_dark', # темная тема
    annotations=[dict(text='соотношение ушедших<br>и лояльных клиентов',
                      x=0.5, y=0.5,
                      font_size=15, showarrow=False)])

fig.show()

через plotly.express 

In [4]:
fig = px.pie(exited_percent,                     
       names =['Остались', 'Ушли'],
       values = 'Exited', 
       color_discrete_sequence = px.colors.sequential.Bluered)

fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

На графике видно соотношение ушедших и лояльных клиентов, осталось больше чем в 3 раза чем ушло, что стоит учесть при работе в ML

## 9.2. Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов.
Опишите распределение и сделайте выводы.


In [5]:
fig = px.histogram(data_frame=churn_data.query('Balance > 2500')['Balance'], x = 'Balance',
             title='распределение баланса пользователей, у которых на счету больше 2500 долларов',
             nbins = 100,
             )
fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

In [50]:
from scipy.stats import norm

x_r = churn_data.query('Balance > 2500')['Balance']

x = np.linspace(x_r.values.min(), x_r.values.max(), len(x_r))
mu, std = norm.fit(x_r.values)
p = norm.pdf(x, mu, std)*x_r.values.max()*130

fig = go.Figure()
fig.add_trace(go.Histogram(nbinsx=100,
                           x=x_r,
                           name='Экспериментальные" данные'))

fig.add_trace(go.Scatter(x=x,
                         y = p,
                         mode = 'lines',
                         name='Теоретическая форма нормального распределения'))
fig.update_layout(
    title="Пример гистограммы на основе нормального распределения",
    title_x = 0.5,
    legend=dict(x=.5,
                xanchor="center",
                orientation="h"),
    margin=dict(l=0, r=0, t=30, b=0))

fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

Распределение баланса пользователей, у которых на счету больше 2'500 долларов, близко к нормальному расределению.

## 9.3. Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? 

Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [284]:
fig = px.box(churn_data,
             y = "Balance",
             x = "Exited",
             points="all",
             notched=True,
             color="Exited", # color by release year
             title="Распределение возраста в разрезе признака оттока",
             labels=['Остались', 'Ушли'],
             )
fig.update_traces(quartilemethod="exclusive")

fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

Распределение баланса на счетах ушедших клиентов выше, также выше их медиана

## 9.4. Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов?

На какую возрастную категорию клиентов стоит обратить внимание банку?

In [285]:
fig = px.box(churn_data,
             y = "Age",
             x = "Exited",
             points="all",
             notched=True,
             color="Exited", # color by release year
             title="Рраспределение возраста в разрезе признака оттока",
             #labels=['Остались', 'Ушли'],
             )
#fig.update_traces(quartilemethod="exclusive")

fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

Потенциальных выбросов больше в группе оставшихся клиентов, особое внимание нужно уделить категории возраста выше 45 лет

## 9.5. Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты.

Добавьте расцветку по признаку оттока клиентов.

Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [286]:
fig = px.density_contour(churn_data,
                         x="CreditScore",
                         y="EstimatedSalary",
                         color="Exited",
                         title="Распределение возраста в разрезе признака оттока",
                         )

fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

In [287]:
fig = px.scatter(churn_data, x="CreditScore",
                 y="EstimatedSalary",
                 color="Exited",
                 size='CreditScore',
                 hover_data=['EstimatedSalary'],
                 title="Распределение возраста в разрезе признака оттока",

                 )
fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

In [288]:
# Build figure
fig = go.Figure()

# Add first scatter trace with medium sized markers
data_0 = churn_data[churn_data.Exited == 0]
data_1 = churn_data[churn_data.Exited == 1]
fig.add_trace(
    go.Scatter(
        mode='markers',
        x= data_0.CreditScore,
        y=data_0.EstimatedSalary,
        
        marker=dict(
            color='LightSkyBlue',
            opacity=0.5,
            size=20,
            line=dict(
                color='MediumPurple',
                width=2
            )
        ),

        name='Exited = 0'
    )
)

# Add second scatter trace with medium sized markers
# and opacity 1.0
fig.add_trace(
    go.Scatter(
        mode='markers',
        x= data_1.CreditScore,
        y=data_1.EstimatedSalary,
         opacity=0.5,
        marker=dict(
            color='Blue',
            size=20,
            line=dict(
                color='SkyBlue',
                width=2
            )
        ),
        name='Exited = 1'
    )
)


# Add trace with large markers
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=[100, 100],
        y=[5000, 100000],
        opacity=0.5,
        marker=dict(
            color=['Blue', 'LightSkyBlue'],
            size=80,
            line=dict(
                color=['SkyBlue', 'MediumPurple'],
                width=8
            )
        ),
        showlegend=False
    )
)

fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

Явной взаимосвязи нет, но лица с очень низким доходом особо не духодят явно, далее в целом паритет

## 9.6. Кто чаще уходит, мужчины или женщины? 

Постройте график, который иллюстрирует это.


### вручную создание таблицы и далее через imshow()

Проблема в том, что разные длины массивов

In [289]:
churn_data[churn_data.Gender == 'Female'].Exited.values.shape

(4543,)

In [290]:
churn_data[churn_data.Gender == 'Male'].Exited.values.shape

(5457,)

Соединяем их с вставкой нан

In [291]:
female_existed = churn_data[churn_data.Gender == 'Female'].Exited
male_existed = churn_data[churn_data.Gender == 'Male'].Exited
gender_existed = [female_existed, male_existed]
gender_existed_comby = np.full([len(gender_existed), len(female_existed)+len(male_existed)], np.nan)

for i in range(gender_existed_comby.shape[1]):
  try: # присваиваем значение если индес тот
    gender_existed_comby[0][i] = gender_existed[0][i]
  except: pass

  try: # присваиваем значение если индес тот
    gender_existed_comby[1][i] = gender_existed[1][i]
  except: pass

In [292]:
gender_existed_comby

array([[ 1.,  0.,  1., ...,  1., nan,  0.],
       [nan, nan, nan, ..., nan,  1., nan]])

Можно скоее еще через merge() по индексу объединить, не пробовал!

In [293]:
fig = px.imshow(gender_existed_comby,
                y= churn_data.Gender.unique(), # ['Female', 'Male'],
                #aspect="auto"
                )
fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

### Тоже через go.Heatmap и pivot_table

In [294]:
pivot = churn_data.pivot_table(
    values='Exited',
    columns='CustomerId',
    index='Gender',
)
pivot

CustomerId,15565701,15565706,15565714,15565779,15565796,15565806,15565878,15565879,15565891,15565996,...,15815534,15815552,15815560,15815615,15815626,15815628,15815645,15815656,15815660,15815690
Gender,,,,,,,,,,,,,,,,,,,,,
Female,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,1.0,0.0,0.0
Male,NaN,1.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,NaN,0.0,0.0,1.0,NaN,0.0,NaN,NaN,NaN


In [295]:
fig = go.Figure(data=go.Heatmap(                 
                   z = pivot,
                   x = pivot.columns,
                   y= churn_data.Gender.unique(), # ['Female', 'Male'],
                   type = 'heatmap',  
                   hoverongaps = False
                   )
)
fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

Женщины уходят немного чаще

## 9.7. Как отток клиентов зависит от числа приобретённых у банка услуг?

 Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [296]:
grouped = (
churn_data
    .groupby(by='NumOfProducts', as_index=False)['Exited']
    .sum()
)


fig = px.bar(grouped,
             x="NumOfProducts",
             y="Exited", color='Exited', 
             height=600,
             title='Как отток клиентов зависит от числа приобретённых у банка услуг?')
fig.show()

Отток клиентов имеет обратную зависимость от числа приобретённых у банка услуг. Чем их меньше, тем отток выше

## 9.8. Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это.

Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?


In [297]:
grouped = (
churn_data
    .groupby(by='IsActiveMember', as_index=False)['Exited']
    .sum()
)


fig = px.bar(grouped,
             x="IsActiveMember",
             y="Exited", color='Exited', 
             height=600,
             title='Как влияет наличие статуса активного клиента на отток клиентов?')
fig.show()

Активные клиенты уходят реже

## 9.9. В какой стране доля ушедших клиентов больше?

Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

In [298]:
grouped = (
churn_data
    .groupby(by='Geography', as_index=False)['Exited']
    .sum()
)
pivot = grouped.pivot_table(
    values='Exited',
    columns='Geography'
)

fig = go.Figure(data=go.Heatmap(x=pivot.columns,
                                y = churn_data.Exited,
                                z = pivot.values
                               ))

fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()

Доля ушедших клиентов больше во Франции и Германии

9.10. Переведите числовой признак CreditScore в категориальный. 


Для этого воспользуйтесь функцией get_credit_score_cat(), которая приведена ниже. Примените её к столбцу CreditScore и создайте новый признак CreditScoreCat — категории кредитного рейтинга.

    def get_credit_score_cat(credit_score):
        if credit_score >= 300 and credit_score < 500:
            return "Very_Poor"
        elif credit_score >= 500 and credit_score < 601:
            return "Poor"
        elif credit_score >= 601 and credit_score < 661:
            return "Fair"
        elif credit_score >= 661 and credit_score < 781:
            return "Good"
        elif credit_score >= 781 and credit_score < 851:
            return "Excellent"
        elif credit_score >= 851:
            return "Top"
        elif credit_score < 300:
            return "Deep"

            
Постройте сводную таблицу, строками которой являются категории кредитного рейтинга (CreditScoreCat), а столбцами — количество лет, в течение которых клиент пользуется услугами банка (Tenure). В ячейках сводной таблицы должно находиться среднее по признаку оттока (Exited) — доля ушедших пользователей.

На основе полученной сводной таблицы постройте тепловую карту с аннотацией. Найдите на тепловой карте категории клиентов, которые уходят чаще всего.

In [299]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"
    
churn_data['CreditScoreCat'] = churn_data['CreditScore'].apply(get_credit_score_cat)

pivot = churn_data.pivot_table(
    values='Exited',
    columns='Tenure',
    index='CreditScoreCat',
)

fig = go.Figure(data=go.Heatmap(                 
                   z = pivot,
                   x = pivot.columns,
                   y= churn_data['CreditScoreCat'].unique(),
                   text=pivot.values.round(2),
                   texttemplate="%{text}",
                   textfont={"size":15}
                   )
)
fig.update_layout(template = 'plotly_dark') # темная тема
fig.show()


Чаще всего уходят категории poor и very_poor почти независимо от срока пользования услугами банка, на втором месте категория Excellent